<a href="https://colab.research.google.com/github/Kewton/kewton.blog.colab/blob/master/declogger.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
_logconfpath = './log.conf'

logconfig = """
[loggers]
keys=root

[handlers]
keys=rotatinghandler, timedrotatinghandler

[formatters]
keys=_formatter

[logger_root]
level=DEBUG
handlers=rotatinghandler, timedrotatinghandler

[handler_rotatinghandler]
class=handlers.RotatingFileHandler
level=DEBUG
formatter=_formatter
args=('./app.log', 'a', 1024*1024, 5, 'utf-8')

[handler_timedrotatinghandler]
class=handlers.TimedRotatingFileHandler
level=ERROR
formatter=_formatter
args=('./rotation.log', 'S', 1024*1024, 5, 'utf-8')


[formatter__formatter]
format=[%(process)d-%(thread)d]-%(asctime)s-[%(filename)s:%(lineno)d]-%(levelname)s-%(message)s
"""

f = open(_logconfpath, 'w')
f.write(logconfig)
f.close()

In [2]:
import logging.config
import datetime
import time
import inspect

def getlogger():
    return logging.getLogger(__name__)

def setfileConfig(_path):
    logging.config.fileConfig(fname=_path)

def declogger(func):
    curframe = inspect.currentframe()
    calframe = inspect.getouterframes(curframe, 2)
    fn = calframe[1][1].split("/")
    filename = fn[len(fn) - 1]
    modulename = fn[len(fn) - 2]
    lineno = calframe[1][2]
    code_context = calframe[1][4]
    funcname = code_context[len(code_context) - 1]
    defaultmessage = "[" + modulename + "." + filename + ":" + str(lineno) + "][" + funcname.strip() + "]:"

    def wrapper(*args, **kwargs):
        sw = StopWatch()
        sw.sw_start()
        _logger = getlogger()
        _logger.debug(defaultmessage + "start:-----")
        kekka = func(*args, **kwargs)
        syorijikan = f"***** 処理時間：{sw.sw_stop()} ***"
        _logger.debug(defaultmessage + "-end-:" + syorijikan)
        return kekka
    wrapper.__name__ = func.__name__
    return wrapper

def edtmessage(message):
    calframe = inspect.getouterframes(inspect.currentframe(), 2)
    fn = calframe[2][1].split("/")
    filename = fn[len(fn) - 1]
    modulename = fn[len(fn) - 2]
    lineno = calframe[1][2]
    return "[" + modulename + "." + filename + ":" + str(lineno) + "][" + calframe[1][3] + "]:" + str(message)

def writedebuglog(message):
  getlogger().debug(edtmessage(message))
  return

def writeinfolog(message):
  getlogger().info(edtmessage(message))
  return

def writeerrorlog(message):
  getlogger().error(edtmessage(message))
  return

class StopWatch:
    def sw_start(self):
        self.__starttime = time.time()
        return
    
    def sw_stop(self):
        return time.time() - self.__starttime


In [3]:
@declogger
def test():
  i = 0
  for j in range(0,10):
    writedebuglog(f"{j}回目の処理を開始します")
    writeinfolog(f"{j}回目の処理を開始します")
    writeerrorlog(f"{j}回目の処理を開始します")
    i = i + j

In [4]:
setfileConfig(_logconfpath)
test()